# Persistence Images using ML for dynamical systems

This notebook is the definition and the training of the NN that predicts PI for the dynamical systems 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from IPython.display import SVG

import gudhi as gd
import gudhi.representations

from tqdm import tqdm
import os

### Load the data

In [ ]:
#data = np.load('data/PI_data_multiple_circle.npz')
data = np.load('data/PI_data_10000_dynamical_alpha.npz')

data_train = data["data_train"]
PI_train = data["PI_train"]
data_test = data["data_test"]
PI_test = data["PI_test"]

In [ ]:
N_sets_train = data_train.shape[0]
N_sets_test = data_test.shape[0]
N_points = data_train.shape[1]
PI_size = int(np.sqrt(PI_train.shape[1]))
dim = 2

In [ ]:
print("N_sets_train : ", N_sets_train)
print("N_sets_test : ", N_sets_test)
print("N_points : ", N_points)
print("PI_size : ", PI_size)

We normalize the PIs

In [ ]:
PI_train /= max(np.max(PI_train), np.max(PI_test))
PI_test /= max(np.max(PI_test), np.max(PI_test))

### Definiton of the NN

In [ ]:
class FullyConnected2(tf.keras.layers.Layer):
    def __init__(self, N_input, N_output):
        super(FullyConnected2, self).__init__()
        self.gamma = self.add_weight(name='gamma',
                                     shape=(N_input, N_output),
                                     initializer="random_normal",
                                     trainable=True)

    def call(self, inputs):
        return tf.einsum("ijk,kl->ijl", inputs, self.gamma)

In [ ]:
#V2

inputs = tf.keras.Input(shape=(N_points, dim))

#x = tf.keras.layers.Dense(30, activation='relu')(inputs)

FC_layer_2_30 = FullyConnected2(dim, 30)
x = FC_layer_2_30(inputs)
x = tf.keras.activations.relu(x)

#x = tf.keras.layers.Dense(35, activation='relu')(x)


x = tf.keras.layers.Dense(20, activation='relu')(x)
#FC_layer_30_20 = FullyConnected2(30, 20)
#x = FC_layer_30_20(x)
#x = tf.keras.activations.relu(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)
#FC_layer_20_10 = FullyConnected2(20, 10)
#x = FC_layer_20_10(x)
#x = tf.keras.activations.relu(x)

Adder = tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x, axis=1),
                               output_shape=(lambda shape:
                                             (shape[0], shape[2])))
x = Adder(x)
#x = tf.keras.activations.relu(x)

#x = tf.keras.layers.Dense(25, activation='relu')(x)

#x = tf.keras.layers.Dense(50, activation='relu')(x)

#x = tf.keras.layers.Dense(100, activation='relu')(x)

#x = tf.keras.layers.Dense(200, activation='relu')(x)

outputs = tf.keras.layers.Dense(PI_size * PI_size, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
adam = tf.keras.optimizers.Adamax(learning_rate=5e-3)  #5e-3 learning_rate optimal d'après moi
model.compile(optimizer=adam, loss="mse")  #contrastive_loss

In [ ]:
model.summary()

SVG(
    tf.keras.utils.model_to_dot(model, show_shapes=True).create(prog='dot',
                                                                format='svg'))
                                                         
#tf.keras.utils.plot_model(model,
#                          to_file='Résultats/Résultats 2/model_multiple_circles.pdf',
#                          show_shapes=True)

### Train the model 

In [ ]:
history = model.fit(data_train[:1000],
                    PI_train[:1000],
                    epochs=100,
                    validation_data=(data_test, PI_test))

We can save the model

In [ ]:
model.save('Saved_Model/model_2_multiple_cricles')

### Study the results to see how the training went 

In [ ]:
prediction = model.predict(data_test)

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.scatter(data_test[i, :, 0], data_test[i, :, 1], s=3)

plt.suptitle('The orbits')
#plt.savefig("Résultats/Résultats 2/multiple_circles.pdf")

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(np.flip(np.reshape(prediction[i], [PI_size, PI_size]), 0),
               vmin=0,
               vmax=1,
               cmap='jet')
    plt.colorbar()
    
plt.suptitle('The corresponding predicted PI')

#plt.savefig("Résultats/Résultats 2/multiple_circles_predicted.pdf")

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(np.flip(np.reshape(PI_test[i], [PI_size, PI_size]), 0),
               vmin=0,
               vmax=1,
               cmap='jet')
    plt.colorbar()
plt.suptitle('The corresponding true PI')

#plt.savefig("Résultats/Résultats 2/multiple_circles_true.pdf")

### Evaluation of the model and plot of the evolution of the loss 

MSE on the test data

In [ ]:
(np.square(prediction - PI_test)).mean(axis=None)

In [ ]:
loss = model.evaluate(data_test, PI_test, verbose=1)

Plot of the evolution of the loss

In [ ]:
history_dict = history.history

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

# "bo" is for "blue dot"
plt.plot(epochs[:], loss[:], 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs[:], val_loss[:], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

#plt.savefig("Résultats/Résultats 2/training_loss.pdf")
plt.show()

## Classifier

In [ ]:
data = np.load('data/PI_data_1000_dynamical_classif.npz')

label_classif_train = data["label_train"]
data_train_classif = data["data_train"]
label_classif_test = data["label_test"]
data_test_classif = data["data_test"]

In [ ]:
inputs = tf.keras.Input(shape=(PI_size * PI_size))

x = tf.keras.layers.Dense(50, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(2)(x)

model_classif = tf.keras.Model(inputs=inputs, outputs=outputs)
model_classif.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])


In [ ]:
PI_train_classif = model.predict(data_train_classif)
PI_test_classif = model.predict(data_test_classif)

In [ ]:
history_classif = model_classif.fit(PI_train_classif,
                                  label_classif_train,
                                  epochs=100,
                                  validation_data=(PI_test_classif, label_classif_test))

In [ ]:
test_loss, test_acc = model_classif.evaluate(PI_test_classif,
                                                label_classif_test,
                                                verbose=2)

print('\nTest accuracy:', test_acc)

Accuracy of the classifier trained on Gudhi : 97%


| Number of point clouds for training 	| Accuracy 	|
|:-----------------------------------:	|:--------:	|
|                 500                 	|   46%    	|
|                 750                 	|   71%    	|
|                 850                 	|   73%    	|
|                 1000                 	|   80%    	|
|                 1500                 	|   78%    	|
|                 2000                 	|      100%    	|
|                 2500                 	|      99%    	|
|                 3000                 	|     100%     	|
|                 5000                	|    100%   |

In [ ]:
acc = [46,71,73,80,78,100,99,100]
number_sets = [500,750,850,1000,1500,2000,2500,3000]

plt.plot(number_sets,acc,'o-')
plt.title("Accuracy of the classifier depending on the number of training sets")
plt.ylabel("acc. of the classifier")
plt.xlabel("Number of point cloud used for training")

plt.annotate("46.0", # this is the text
                 (500,46), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(11,0), # distance from text to points (x,y)
                 ha='left') # horizontal alignment can be left, right or center

plt.annotate("71.0", # this is the text
                 (750,71), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-27,-3), # distance from text to points (x,y)
                 ha='left') # horizontal alignment can be left, right or center




for x,y in zip(number_sets[2:],acc[2:]):

    label = "{:.1f}".format(y)

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,-14), # distance from text to points (x,y)
                 ha='left') # horizontal alignment can be left, right or center

plt.savefig("Résultats/Résultats 4/acc_number_training_set_syst_dyn.pdf")
